In [4]:
# Import all necessary libraries
import os
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.ai.ml import automl, Input, MLClient, command
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Data
from azure.ai.ml.automl import (
    classification,
    ClassificationPrimaryMetrics,
    ClassificationModels,
)


In [5]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
except Exception as ex:
    print("Authentication failed:", ex)

# Automatically loads configuration from config.json or config.yaml
ml_client = MLClient.from_config(credential=credential)

# Verify the connection
workspace = ml_client.workspaces.get(ml_client.workspace_name)
print(workspace.location, ":", workspace.resource_group)


Found the config file in: ./config.json


uksouth : ML_Project


In [6]:
training_data_input = Input(
    type=AssetTypes.MLTABLE, path="azureml:credit_card:1"
)

In [7]:
from azure.ai.ml import automl

classification_job = automl.classification(
    compute="Nmasuud1",
    experiment_name="credit_card_data",
    training_data=training_data_input,
    target_column_name="default payment next month",  
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)
classification_job.set_limits(
    timeout_minutes=30, 
    trial_timeout_minutes=10, 
    max_trials=5,
    enable_early_termination=True,
)


In [8]:
# Submit the classification job
returned_job = ml_client.jobs.create_or_update(classification_job)

# Print the Studio URL for manual monitoring
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

# Stream logs for real-time updates
print("Waiting for the job to complete...")
ml_client.jobs.stream(returned_job.name)


Monitor your job at https://ml.azure.com/runs/lemon_reggae_1sxfyxv12h?wsid=/subscriptions/43be7692-0d67-4524-a6de-08c117967d94/resourcegroups/ML_Project/workspaces/mlproject&tid=104704fd-f711-4575-99d5-8a6588951c90
Waiting for the job to complete...
RunId: lemon_reggae_1sxfyxv12h
Web View: https://ml.azure.com/runs/lemon_reggae_1sxfyxv12h?wsid=/subscriptions/43be7692-0d67-4524-a6de-08c117967d94/resourcegroups/ML_Project/workspaces/mlproject

Execution Summary
RunId: lemon_reggae_1sxfyxv12h
Web View: https://ml.azure.com/runs/lemon_reggae_1sxfyxv12h?wsid=/subscriptions/43be7692-0d67-4524-a6de-08c117967d94/resourcegroups/ML_Project/workspaces/mlproject



In [9]:
import mlflow
# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = ml_client.workspaces.get(
    name=ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

azureml://uksouth.api.azureml.ms/mlflow/v1.0/subscriptions/43be7692-0d67-4524-a6de-08c117967d94/resourceGroups/ML_Project/providers/Microsoft.MachineLearningServices/workspaces/mlproject


In [10]:
# Set the MLFLOW TRACKING URI

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print("\nCurrent tracking uri: {}".format(mlflow.get_tracking_uri()))

from mlflow.tracking.client import MlflowClient
from mlflow.artifacts import download_artifacts

# Initialize MLFlow client
mlflow_client = MlflowClient()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service


In [11]:
job_name = returned_job.name

# Example if providing an specific Job name/ID
# job_name = "b4e95546-0aa1-448e-9ad6-002e3207b4fc"

# Get the parent run
mlflow_parent_run = mlflow_client.get_run(job_name)

print("Parent Run: ")
print(mlflow_parent_run)

Parent Run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.7806349653079421,
 'AUC_micro': 0.8826162500000001,
 'AUC_weighted': 0.7806349653079422,
 'accuracy': 0.8204,
 'average_precision_score_macro': 0.7348484716693705,
 'average_precision_score_micro': 0.8724669200422046,
 'average_precision_score_weighted': 0.8333295805443417,
 'balanced_accuracy': 0.6583507368021954,
 'f1_score_macro': 0.6834116942936708,
 'f1_score_micro': 0.8204,
 'f1_score_weighted': 0.7995623094560103,
 'log_loss': 0.4287022823628952,
 'matthews_correlation': 0.40314903424602333,
 'norm_macro_recall': 0.31670147360439066,
 'precision_score_macro': 0.7567052283061886,
 'precision_score_micro': 0.8204,
 'precision_score_weighted': 0.8037923336942328,
 'recall_score_macro': 0.6583507368021954,
 'recall_score_micro': 0.8204,
 'recall_score_weighted': 0.8204,
 'weighted_accuracy': 0.9055511810288003}, params={}, tags={'automl_best_child_run_id': 'lemon_reggae_1sxfyxv12h_3',
 'fit_time_000': '0.5935857999999999;8.48

In [12]:
# Print parent run tags. 'automl_best_child_run_id' tag should be there.
print(mlflow_parent_run.data.tags)

{'model_explain_run': 'best_run', 'pipeline_id_000': '5dfac790c5c209f98a1da2dc1c7fb76f0397324f;c7af0367625be6ac5c2fecbfc72ed444cb7a2111;799d2168db11fc19b9e1c6c1df62f8981ad39fe9;__AutoML_Ensemble__;__AutoML_Stack_Ensemble__', 'score_000': '0.8196;0.8153666666666666;0.7505666666666666;0.8204;0.8194666666666667', 'predicted_cost_000': '0;0;0.5;0;0', 'fit_time_000': '0.5935857999999999;8.4867412;0.9193876000000001;3;9', 'training_percent_000': '100;100;100;100;100', 'iteration_000': '0;1;2;3;4', 'run_preprocessor_000': 'MaxAbsScaler;MaxAbsScaler;MaxAbsScaler;;', 'run_algorithm_000': 'LightGBM;XGBoostClassifier;ExtremeRandomTrees;VotingEnsemble;StackEnsemble', 'automl_best_child_run_id': 'lemon_reggae_1sxfyxv12h_3', 'model_explain_best_run_child_id': 'lemon_reggae_1sxfyxv12h_3', 'mlflow.rootRunId': 'lemon_reggae_1sxfyxv12h', 'mlflow.runName': 'lemon_reggae_1sxfyxv12h', 'mlflow.user': 'Nimatallahi Massud'}


In [13]:

# Attempt to get the best model's child run using 'automl_best_child_run_id' tag
best_child_run_id = mlflow_parent_run.data.tags.get("automl_best_child_run_id")

if best_child_run_id:
    print("Found best child run id: ", best_child_run_id)
    best_run = mlflow_client.get_run(best_child_run_id)
else:
    print("'automl_best_child_run_id' tag not found. Searching for best child run manually...")

    # Search for child runs of the parent run
    parent_run_id = mlflow_parent_run.info.run_id
    child_runs = mlflow.search_runs([parent_run_id])

    if len(child_runs) > 0:
        # Sort child runs based on a metric, e.g., accuracy or any performance measure you care about
        best_run = child_runs.iloc[child_runs['metrics.accuracy'].idxmax()]
        best_child_run_id = best_run.run_id
        print(f"Found best child run id: {best_child_run_id} with accuracy: {best_run['metrics.accuracy']}")
    else:
        print("No child runs found for the given parent run.")

# If a best run is found, you can proceed to work with it
if best_child_run_id:
    best_run = mlflow_client.get_run(best_child_run_id)
    print("Best child run details: ")
    print(best_run)
else:
    print("Unable to find a best child run.")

Found best child run id:  lemon_reggae_1sxfyxv12h_3
Best child run details: 
<Run: data=<RunData: metrics={'AUC_macro': 0.7806349653079421,
 'AUC_micro': 0.8826162500000001,
 'AUC_weighted': 0.7806349653079422,
 'accuracy': 0.8204,
 'average_precision_score_macro': 0.7348484716693705,
 'average_precision_score_micro': 0.8724669200422046,
 'average_precision_score_weighted': 0.8333295805443417,
 'balanced_accuracy': 0.6583507368021954,
 'f1_score_macro': 0.6834116942936708,
 'f1_score_micro': 0.8204,
 'f1_score_weighted': 0.7995623094560103,
 'log_loss': 0.4287022823628952,
 'matthews_correlation': 0.40314903424602333,
 'norm_macro_recall': 0.31670147360439066,
 'precision_score_macro': 0.7567052283061886,
 'precision_score_micro': 0.8204,
 'precision_score_weighted': 0.8037923336942328,
 'recall_score_macro': 0.6583507368021954,
 'recall_score_micro': 0.8204,
 'recall_score_weighted': 0.8204,
 'weighted_accuracy': 0.9055511810288003}, params={}, tags={'mlflow.parentRunId': 'lemon_regga

In [14]:
# Check the status of the parent run
print("Parent run status: ", mlflow_parent_run.info.status)


Parent run status:  FINISHED


In [15]:
# Force refresh the parent run info
mlflow_parent_run = mlflow_client.get_run(mlflow_parent_run.info.run_id)
print("Updated Parent run status: ", mlflow_parent_run.info.status)


Updated Parent run status:  FINISHED


In [16]:
# Get the best model's child run

best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print("Found best child run id: ", best_child_run_id)

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

Found best child run id:  lemon_reggae_1sxfyxv12h_3
Best child run: 
<Run: data=<RunData: metrics={'AUC_macro': 0.7806349653079421,
 'AUC_micro': 0.8826162500000001,
 'AUC_weighted': 0.7806349653079422,
 'accuracy': 0.8204,
 'average_precision_score_macro': 0.7348484716693705,
 'average_precision_score_micro': 0.8724669200422046,
 'average_precision_score_weighted': 0.8333295805443417,
 'balanced_accuracy': 0.6583507368021954,
 'f1_score_macro': 0.6834116942936708,
 'f1_score_micro': 0.8204,
 'f1_score_weighted': 0.7995623094560103,
 'log_loss': 0.4287022823628952,
 'matthews_correlation': 0.40314903424602333,
 'norm_macro_recall': 0.31670147360439066,
 'precision_score_macro': 0.7567052283061886,
 'precision_score_micro': 0.8204,
 'precision_score_weighted': 0.8037923336942328,
 'recall_score_macro': 0.6583507368021954,
 'recall_score_micro': 0.8204,
 'recall_score_weighted': 0.8204,
 'weighted_accuracy': 0.9055511810288003}, params={}, tags={'mlflow.parentRunId': 'lemon_reggae_1sxfyx

In [17]:
best_run.data.metrics

{'weighted_accuracy': 0.9055511810288003,
 'recall_score_weighted': 0.8204,
 'recall_score_micro': 0.8204,
 'matthews_correlation': 0.40314903424602333,
 'accuracy': 0.8204,
 'AUC_micro': 0.8826162500000001,
 'f1_score_weighted': 0.7995623094560103,
 'average_precision_score_weighted': 0.8333295805443417,
 'f1_score_macro': 0.6834116942936708,
 'norm_macro_recall': 0.31670147360439066,
 'precision_score_macro': 0.7567052283061886,
 'precision_score_micro': 0.8204,
 'AUC_macro': 0.7806349653079421,
 'precision_score_weighted': 0.8037923336942328,
 'log_loss': 0.4287022823628952,
 'average_precision_score_macro': 0.7348484716693705,
 'balanced_accuracy': 0.6583507368021954,
 'AUC_weighted': 0.7806349653079422,
 'recall_score_macro': 0.6583507368021954,
 'f1_score_micro': 0.8204,
 'average_precision_score_micro': 0.8724669200422046}

In [18]:
# import required libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
    ProbeSettings,
)
from azure.ai.ml.constants import ModelType

In [19]:
import datetime

# Generate a valid online endpoint name
experiment_name = "creditcarddata"  # Ensure this starts with a letter and has no spaces or special characters
timestamp = datetime.datetime.now().strftime("%m%d%H%M%S")  # Removes microseconds for simplicity
online_endpoint_name = f"{experiment_name}-{timestamp}"

# Create an online endpoint with the unique name
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for the credit card data experiment.",
    auth_mode="key",  # Default authentication mode
)

# Print the generated endpoint name
print(f"Generated online endpoint name: {online_endpoint_name}")


Generated online endpoint name: creditcarddata-0112133933


In [20]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://creditcarddata-0112133933.uksouth.inference.ml.azure.com/score', 'openapi_uri': 'https://creditcarddata-0112133933.uksouth.inference.ml.azure.com/swagger.json', 'name': 'creditcarddata-0112133933', 'description': 'Online endpoint for the credit card data experiment.', 'tags': {}, 'properties': {'createdBy': 'Nimatallahi Massud', 'createdAt': '2025-01-12T13:39:38.644542+0000', 'lastModifiedAt': '2025-01-12T13:39:38.644542+0000', 'azureml.onlineendpointid': '/subscriptions/43be7692-0d67-4524-a6de-08c117967d94/resourcegroups/ml_project/providers/microsoft.machinelearningservices/workspaces/mlproject/onlineendpoints/creditcarddata-0112133933', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/43be7692-0d67-4524-a6de-08c117967d94/providers/Microsoft.MachineLearningServices/locations/uksouth/mfeOperationsStatus/oeidp:afb0ba33-1525-4e26-8e44-aed56d3b908b:81

In [33]:
# Best child run ID
best_child_run_id = "lemon_reggae_1sxfyxv12h_3"

# Construct model path
artifact_uri = f"azureml://jobs/{best_child_run_id}/outputs/artifacts"
model_path = f"{artifact_uri}/outputs/mlflow-model/"

# Define the model
model = Model(
    path=model_path,
    name="creditcard-score-model",
    description="Classification model for credit card score detection",
    type=AssetTypes.MLFLOW_MODEL,
)

# Register the model
registered_model = ml_client.models.create_or_update(model)
print(f"Registered model: {registered_model.name}, version: {registered_model.version}")
registered_model.id


Registered model: creditcard-score-model, version: 4


'/subscriptions/43be7692-0d67-4524-a6de-08c117967d94/resourceGroups/ML_Project/providers/Microsoft.MachineLearningServices/workspaces/mlproject/models/creditcard-score-model/versions/4'

In [38]:
deployment = ManagedOnlineDeployment(
    name="creditcard-score-deploy",
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS2_v2",
    instance_count=1,
    liveness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=2000,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

In [39]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

Instance type Standard_DS2_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint creditcarddata-0112133933 exists


......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'creditcarddata-0112133933', 'type': 'Managed', 'name': 'creditcard-score-deploy', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/43be7692-0d67-4524-a6de-08c117967d94/providers/Microsoft.MachineLearningServices/locations/uksouth/mfeOperationsStatus/odidp:afb0ba33-1525-4e26-8e44-aed56d3b908b:b2df770a-dbcf-4187-82a8-225d9dab0cf0?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/43be7692-0d67-4524-a6de-08c117967d94/resourceGroups/ML_Project/providers/Microsoft.MachineLearningServices/workspaces/mlproject/onlineEndpoints/creditcarddata-0112133933/deployments/creditcard-score-deploy', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/nmasuud1/code/Users/Nmasuud', 'creation_context': <azure.ai.ml._restclient.v2023_04_

In [40]:
# creditcard-score deployment to take 100% traffic
endpoint.traffic = {"creditcard-score-deploy": 100}
ml_client.begin_create_or_update(endpoint)
